In [2]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage


In [44]:
import pandas as pd
train_dataset = pd.read_csv('/home/ashish/llama_inference/clinical-outcome-prediction/saved_files/DIA_GROUPS_3_DIGITS_adm_train.csv')
train_dataset.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33994 entries, 0 to 33993
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 33994 non-null  int64 
 1   text               33994 non-null  object
 2   long_texts         33994 non-null  object
 3   short_texts        33994 non-null  object
 4   discharge_summary  33994 non-null  object
 5   short_codes        33994 non-null  object
dtypes: int64(1), object(5)
memory usage: 1.6+ MB


In [4]:
icd_9_df = pd.read_csv('/home/ashish/llama_inference/physionet.org/files/mimiciii/1.4/D_ICD_DIAGNOSES.csv')
icd_9_df.head()

,ROW_ID,ICD9_CODE,SHORT_TITLE,LONG_TITLE
0,174,01166,TB pneumonia-oth test,"Tuberculous pneumonia [any form], tubercle bac..."
1,175,01170,TB pneumothorax-unspec,"Tuberculous pneumothorax, unspecified"
2,176,01171,TB pneumothorax-no exam,"Tuberculous pneumothorax, bacteriological or h..."
3,177,01172,TB pneumothorx-exam unkn,"Tuberculous pneumothorax, bacteriological or h..."
4,178,01173,TB pneumothorax-micro dx,"Tuberculous pneumothorax, tubercle bacilli fou..."


In [5]:
# icd_9_df[icd_9_df.LONG_TITLE == 'Ankylosing spondylosis with ligament disruption of T8-T9']

In [6]:
# print(train_dataset.iloc[2].discharge_summary)

In [7]:
# prompt_template= f"""Task: Convert clinical diagnoses to ICD-9 long titles

# Instructions:
# 1. Read the given clinical note carefully.
# 2. Identify all medical conditions mentioned.
# 3. Convert each condition to its corresponding ICD-9 long title.
# 4. Create a Python list containing these long titles as strings.
# 5. Output only the Python list, without any additional text or explanations.

# Rules:
# - Use exact ICD-9 long titles, do not abbreviate or modify them.
# - Do not include ICD-9 codes or any explanatory text.
# - The output should be a valid Python list containing only strings.

# Example:
# Input: 'Acute appendicitis, Type 2 diabetes'
# Output: ["Acute appendicitis without mention of peritonitis", "Diabetes mellitus without mention of complication, type II or unspecified type, not stated as uncontrolled"]

# Now, convert the following clinical note to a Python list of ICD-9 long titles:

# Clinical note:

# {clinical_note}


# Output:"""


In [8]:
kg_prompt = """Analyze the symptom-to-disease mapping data:

1. Identify the single symptom.
2. For each unique disease code:
   a. Sum weights across all occurrences.
3. Rank unique codes by total weight.
4. Select the top 5 unique codes:
   - Start with the highest-weighted code
   - For each subsequent code, include ONLY if NOT already in the list
   - Continue until EXACTLY 5 DIFFERENT codes are selected
5. Output ONLY this JSON, with EXACTLY 5 UNIQUE codes:
   {"Symptom": ["code1", "code2", "code3", "code4", "code5"]}

Produce ONLY the JSON. Double-check that all 5 codes are different before outputting."""

In [9]:
codes = """Symptom: underweight, Disease: R635 - Abnormal weight gain - 42.30903, Weight: 1665
Symptom: underweight, Disease: R635 - Abnormal weight gain - 40.921864, Weight: 857
Symptom: underweight, Disease: O481 - Prolonged pregnancy - 61.402317, Weight: 561
Symptom: underweight, Disease: Z6845 - Body mass index (BMI) 70 or greater, adult - 34.038483, Weight: 286
Symptom: underweight, Disease: O481 - Prolonged pregnancy - 57.598442, Weight: 165
Symptom: underweight, Disease: O252 - Malnutrition in childbirth - 27.399006, Weight: 155
Symptom: underweight, Disease: R635 - Abnormal weight gain - 46.521675, Weight: 114
Symptom: underweight, Disease: O481 - Prolonged pregnancy - 63.16733, Weight: 91
Symptom: underweight, Disease: R635 - Abnormal weight gain - 44.510216, Weight: 81
Symptom: underweight, Disease: R51 - Headache - 65.78746, Weight: 77
Symptom: underweight, Disease: E875 - Hyperkalemia - 39.07981, Weight: 73
Symptom: underweight, Disease: R636 - Underweight - 52.455105, Weight: 73
Symptom: underweight, Disease: E59 - Dietary selenium deficiency - 34.213947, Weight: 73
Symptom: underweight, Disease: B483 - Geotrichosis - 31.458736, Weight: 70
Symptom: underweight, Disease: Z526 - Liver donor - 44.89, Weight: 66
Symptom: underweight, Disease: R630 - Anorexia - 25.20142, Weight: 64
Symptom: underweight, Disease: F5001 - Anorexia nervosa, restricting type - 16.134495, Weight: 62
Symptom: underweight, Disease: R030 - Elevated blood-pressure reading, without diagnosis of hypertension - 27.44202, Weight: 55
Symptom: underweight, Disease: O481 - Prolonged pregnancy - 39.45047, Weight: 53
Symptom: underweight, Disease: E300 - Delayed puberty - 46.460373, Weight: 49
Symptom: underweight, Disease: A982 - Kyasanur Forest disease - 36.587307, Weight: 48
Symptom: underweight, Disease: G14 - Postpolio syndrome - 47.895393, Weight: 44
Symptom: underweight, Disease: Z370 - Single live birth - 41.956947, Weight: 42
Symptom: underweight, Disease: R100 - Acute abdomen - 56.566902, Weight: 41
Symptom: underweight, Disease: R635 - Abnormal weight gain - 51.896732, Weight: 39
Symptom: underweight, Disease: A790 - Trench fever - 34.42422, Weight: 36
Symptom: underweight, Disease: Z3A34 - 34 weeks gestation of pregnancy - 28.03623, Weight: 35
Symptom: underweight, Disease: Y9311 - Activity, swimming - 50.684853, Weight: 33
Symptom: underweight, Disease: K901 - Tropical sprue - 44.801044, Weight: 33
Symptom: underweight, Disease: K141 - Geographic tongue - 18.553707, Weight: 33
Symptom: underweight, Disease: Z597 - Insufficient social insurance and welfare support - 43.301476, Weight: 31
Symptom: underweight, Disease: J660 - Byssinosis - 34.828552, Weight: 30
Symptom: underweight, Disease: B04 - Monkeypox - 40.526337, Weight: 30
Symptom: underweight, Disease: O754 - Other complications of obstetric surgery and procedures - 42.495674, Weight: 27
Symptom: underweight, Disease: E119 - Type 2 diabetes mellitus without complications - 32.054657, Weight: 27
Symptom: underweight, Disease: R100 - Acute abdomen - 66.76583, Weight: 27
Symptom: underweight, Disease: B04 - Monkeypox - 40.23687, Weight: 26
Symptom: underweight, Disease: M818 - Other osteoporosis without current pathological fracture - 27.087894, Weight: 25
Symptom: underweight, Disease: K140 - Glossitis - 28.011814, Weight: 24
Symptom: underweight, Disease: Z3A34 - 34 weeks gestation of pregnancy - 23.637177, Weight: 23
Symptom: underweight, Disease: M272 - Inflammatory conditions of jaws - 52.09495, Weight: 22
Symptom: underweight, Disease: O252 - Malnutrition in childbirth - 37.7907, Weight: 22
Symptom: underweight, Disease: K901 - Tropical sprue - 38.53381, Weight: 21
Symptom: underweight, Disease: E65 - Localized adiposity - 25.894415, Weight: 21
Symptom: underweight, Disease: E65 - Localized adiposity - 25.45605, Weight: 21
Symptom: underweight, Disease: F0150 - Vascular dementia without behavioral disturbance - 29.198895, Weight: 20
Symptom: underweight, Disease: B701 - Sparganosis - 42.460114, Weight: 19
Symptom: underweight, Disease: M6284 - Sarcopenia - 23.786839, Weight: 18
Symptom: underweight, Disease: M4040 - Postural lordosis, site unspecified - 53.392548, Weight: 18
Symptom: underweight, Disease: A982 - Kyasanur Forest disease - 35.135307, Weight: 17
"""

kg_prompt = kg_prompt + codes

In [10]:
train_dataset.head()

,id,text,long_texts,short_texts,discharge_summary,short_codes
0,159643,CHIEF COMPLAINT: \n\nPRESENT ILLNESS: This 60 ...,"Aortic valve disorders,Congestive heart failur...","Aortic valve disorder,CHF NOS,Hyperpotassemia,...",Admission Date: [**2141-3-13**] Dischar...,"4241,4280,2767,9971,42732,41401,412,V4582"
1,147171,CHIEF COMPLAINT: Substernal Chest Pain\n\nPRES...,Acute myocardial infarction of other anterior ...,"AMI anterior wall, init,Ac systolic hrt failur...",Admission Date: [**2102-9-26**] ...,"41011,42821,9971,4271,41401,4280,42731,45829"
2,199961,CHIEF COMPLAINT: \n\nPRESENT ILLNESS: The pati...,"Ankylosing spondylitis,Hypertensive chronic ki...","Ankylosing spondylitis,Hyp kid NOS w cr kid V,...",Admission Date: [**2115-6-29**] Dischar...,"7200,40391,8052,8471,E8859,78057,2859,25060"
3,136812,CHIEF COMPLAINT: \n\nPRESENT ILLNESS: This is ...,Obstructive chronic bronchitis with (acute) ex...,"Obs chr bronc w(ac) exac,Pneumonia, organism N...",Admission Date: [**2106-4-14**] Dischar...,"49121,486,2800,42789,42611,43889,72989,2449"
4,175700,CHIEF COMPLAINT: s/p rollover MVC with prolong...,"Closed fracture of shaft of fibula with tibia,...","Fx shaft fib w tib-clos,Pneumococcal pneumonia...",Admission Date: [**2159-2-9**] D...,"82322,481,86121,5180,2851,81322,8072,E8160,883..."


In [11]:
from mistralai.models.jobs import TrainingParameters
from mistralai.client import MistralClient
import pandas as pd
import os
import json

model= "open-mistral-7b" 
MISTRAL_API_KEY =  "WRhxY4qx7jsun5iYThrdS3Dk4dubsjnV"
client = MistralClient(api_key=MISTRAL_API_KEY)

In [12]:
train_dataset.id.nunique()

33994

In [13]:
from tqdm import tqdm

In [14]:
# os.mkdir('train_symptoms')

In [15]:
train_dataset = train_dataset.iloc[20650:].reset_index(drop= True)
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13344 entries, 0 to 13343
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 13344 non-null  int64 
 1   text               13344 non-null  object
 2   long_texts         13344 non-null  object
 3   short_texts        13344 non-null  object
 4   discharge_summary  13344 non-null  object
 5   short_codes        13344 non-null  object
dtypes: int64(1), object(5)
memory usage: 625.6+ KB


In [16]:
all_symptoms = {}

In [1]:
# from tqdm import tqdm
# for i,row in tqdm(train_dataset.iterrows()) : 
#     sys_prompt_template =  """
# You are an AI assistant analyzing clinical discharge summary notes from the MIMIC III dataset. Focus on these key sections:

# >>  Symptoms: It is a subjective indication of a disease or medical condition that is experienced and reported by the patient. Unlike signs, which are objective and can be observed or measured by healthcare professionals,symptoms are personal experiences that only the individual can perceive.

# >> Instructions:
# 1. Carefully read the provided clinical discharge summary note.
# 2. Categorize the information into the corresponding sections in the JSON format.
# 3. Present your analysis in this JSON format:

# {
#   "Symptoms": []
# }

# >> Important:
# - Keep only the Symptoms as a key in JSON format.
# - Keep each symptoms clear and 2-3 words only,  adhering to medical ontology standards.
# - Extract key information concisely, focusing on medically relevant details.
# - Do NOT add any information that is not explicitly stated in the input text.

# Now, analyze and categorize the following clinical admission note using the specified JSON format:

# """
#     clinical_note = f"{row['discharge_summary']}"
#     sys_prompt_template += clinical_note
#     prompt_template = sys_prompt_template
    
#     chat_response = client.chat(
#         model=model,
#         max_tokens=1000,
#         temperature=0.1,
#         messages=[ChatMessage(role="user", content=prompt_template)]
#     )
#     all_symptoms[row.id] = chat_response.choices[0].message.content
#     try : 
#         with open(f'train_symptoms/{row.id}.json', 'w') as f:
#             json.dump(json.loads(chat_response.choices[0].message.content), f)
#     except : 
#         continue
#     # print(chat_response.choices[0].message.content)
#     # break
    
    

In [4]:
# pip install ipywidgets
import os

In [6]:
os.listdir('/home/ashish/llama_inference/train_symptoms')[0]

'159643.json'

In [17]:
train_dataset.head()


,id,text,long_texts,short_texts,discharge_summary,short_codes
0,198736,CHIEF COMPLAINT: Chest pain\n\nPRESENT ILLNESS...,Acute myocardial infarction of other anterior ...,"AMI anterior wall, init,Cardiogenic shock,Comp...",Admission Date: [**2166-2-13**] ...,"41011,78551,99672,9971,4271,99859,0389,99592,5..."
1,197289,CHIEF COMPLAINT: \n\nPRESENT ILLNESS: The pati...,Acute myocardial infarction of other lateral w...,"AMI lateral NEC, initial,Hematoma complic proc...",Admission Date: [**2179-4-2**] Discharge ...,"41051,99812,2851,41401,E8788,6202,4940,2449,4019"
2,107725,CHIEF COMPLAINT: Chest pain.\n\nPRESENT ILLNES...,Coronary atherosclerosis of native coronary ar...,"Crnry athrscl natve vssl,Angina pectoris NEC/N...",Unit No: [**Numeric Identifier 67586**]\nAdmi...,"41401,4139,42731,25000,79439,42789,2939"
3,191355,"CHIEF COMPLAINT: Fevers, nausea, vomiting\n\nP...",Infection and inflammatory reaction due to oth...,"React-oth genitourin dev,Septicemia NOS,Severe...",Admission Date: [**2157-10-17**] ...,"99665,0389,99592,5990,0413,42822,4280,27652,42..."
4,170875,CHIEF COMPLAINT: Dyspnea on exertion\n\nPRESEN...,"Diastolic heart failure, unspecified,Aortic va...","Diastolc hrt failure NOS,Aortic valve disorder...",Admission Date: [**2192-12-23**] ...,"42830,4241,4240,41401,4019,25000,2859,27800,V4..."


In [62]:
symptoms_data = []

In [63]:
for i in os.listdir('/home/ashish/llama_inference/train_symptoms'): 
    subject_id = i.split('.')[0]
    with open(f"/home/ashish/llama_inference/train_symptoms/{i}", 'r') as f:
        content = f.read().strip()  # read and strip any whitespace
        if not content:
            print(f"The file {i} is empty.")
        else:
            data = json.loads(content)
            data['subject_id'] = int(subject_id)
            symptoms_data.append(data)
    

The file 159643.json is empty.
The file 106275.json is empty.
The file 125351.json is empty.
The file 112341.json is empty.
The file 179346.json is empty.
The file 186728.json is empty.
The file 128870.json is empty.
The file 140335.json is empty.
The file 130939.json is empty.
The file 134698.json is empty.
The file 197626.json is empty.
The file 190950.json is empty.
The file 124063.json is empty.
The file 118110.json is empty.
The file 125166.json is empty.
The file 170418.json is empty.
The file 139248.json is empty.
The file 110302.json is empty.
The file 168628.json is empty.
The file 130680.json is empty.
The file 196002.json is empty.
The file 136001.json is empty.
The file 148206.json is empty.
The file 196535.json is empty.
The file 105009.json is empty.
The file 103909.json is empty.
The file 101005.json is empty.
The file 112174.json is empty.
The file 146336.json is empty.
The file 143178.json is empty.
The file 179269.json is empty.
The file 143955.json is empty.
The file

In [64]:
symptoms_df = pd.DataFrame(symptoms_data)
symptoms_df.head()

,Symptoms,subject_id
0,"[Substernal Chest Pain, Sharp Pain, Crescendoe...",147171
1,[Back pain],199961
2,"[Shortness of breath, Cough, Occasional nonpro...",136812
3,"[left arm pain, left leg pain, pulmonary contu...",175700
4,"[Palpitations, Shortness of breath, Sore throat]",193486


In [65]:
symptoms_df.columns = ['Symptoms', 'id']

In [66]:
train_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33994 entries, 0 to 33993
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 33994 non-null  int64 
 1   text               33994 non-null  object
 2   long_texts         33994 non-null  object
 3   short_texts        33994 non-null  object
 4   discharge_summary  33994 non-null  object
 5   short_codes        33994 non-null  object
dtypes: int64(1), object(5)
memory usage: 1.6+ MB


In [67]:
symptoms_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33684 entries, 0 to 33683
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Symptoms  33684 non-null  object
 1   id        33684 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 526.4+ KB


In [68]:
merged_df = pd.merge(symptoms_df, train_dataset, how = 'inner', on = 'id')
merged_df.head()

,Symptoms,id,text,long_texts,short_texts,discharge_summary,short_codes
0,"[Substernal Chest Pain, Sharp Pain, Crescendoe...",147171,CHIEF COMPLAINT: Substernal Chest Pain\n\nPRES...,Acute myocardial infarction of other anterior ...,"AMI anterior wall, init,Ac systolic hrt failur...",Admission Date: [**2102-9-26**] ...,"41011,42821,9971,4271,41401,4280,42731,45829"
1,[Back pain],199961,CHIEF COMPLAINT: \n\nPRESENT ILLNESS: The pati...,"Ankylosing spondylitis,Hypertensive chronic ki...","Ankylosing spondylitis,Hyp kid NOS w cr kid V,...",Admission Date: [**2115-6-29**] Dischar...,"7200,40391,8052,8471,E8859,78057,2859,25060"
2,"[Shortness of breath, Cough, Occasional nonpro...",136812,CHIEF COMPLAINT: \n\nPRESENT ILLNESS: This is ...,Obstructive chronic bronchitis with (acute) ex...,"Obs chr bronc w(ac) exac,Pneumonia, organism N...",Admission Date: [**2106-4-14**] Dischar...,"49121,486,2800,42789,42611,43889,72989,2449"
3,"[left arm pain, left leg pain, pulmonary contu...",175700,CHIEF COMPLAINT: s/p rollover MVC with prolong...,"Closed fracture of shaft of fibula with tibia,...","Fx shaft fib w tib-clos,Pneumococcal pneumonia...",Admission Date: [**2159-2-9**] D...,"82322,481,86121,5180,2851,81322,8072,E8160,883..."
4,"[Palpitations, Shortness of breath, Sore throat]",193486,CHIEF COMPLAINT: Shortness of breath\n\nPRESEN...,"Other specified cardiac dysrhythmias,End stage...","Cardiac dysrhythmias NEC,End stage renal disea...",Admission Date: [**2136-10-4**] ...,"42789,5856,6822,6164,42830,2761,2869,4160,0411..."


In [69]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33684 entries, 0 to 33683
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Symptoms           33684 non-null  object
 1   id                 33684 non-null  int64 
 2   text               33684 non-null  object
 3   long_texts         33684 non-null  object
 4   short_texts        33684 non-null  object
 5   discharge_summary  33684 non-null  object
 6   short_codes        33684 non-null  object
dtypes: int64(1), object(6)
memory usage: 1.8+ MB


In [70]:
merged_df.to_csv('symptoms_train.csv')